---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    readonly/train.csv - the training set (all tickets issued 2004-2011)
    readonly/test.csv - the test set (all tickets issued 2012-2016)
    readonly/addresses.csv & readonly/latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `readonly/train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `readonly/test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import MinMaxScaler
%matplotlib notebook

In [25]:
def blight_model():
    df_train = pd.read_csv('readonly/train.csv', encoding='ISO-8859-1', low_memory=False)
    df_test = pd.read_csv('readonly/test.csv')
    pd.options.mode.chained_assignment = None 
    
    def preprocess(df, isTrain=True):
        df_addresses = pd.read_csv('readonly/addresses.csv')
        df_latlons = pd.read_csv('readonly/latlons.csv')
        
        # Columns to retain
        columns = ['ticket_id', 'ticket_issued_date', 'hearing_date', 'fine_amount', 'admin_fee', 
                   'state_fee', 'late_fee', 'discount_amount',
                   'judgment_amount', 'lat', 'lon']
        
        # Merge data to address and latlons
        df_merge = (pd.merge(df, df_addresses, how='inner', left_on='ticket_id', right_on='ticket_id')
                      .merge(df_latlons, left_on='address', right_on='address')
                      .loc[:,columns]
                      .set_index('ticket_id'))
        
        # Calculate difference between hearing and ticket issued date
        df_merge['hearing_date'] = pd.to_datetime(df_merge['hearing_date'])
        df_merge['ticket_issued_date'] = pd.to_datetime(df_merge['ticket_issued_date'])
        df_merge['diff_date'] = df_merge['hearing_date'].sub(df_merge['ticket_issued_date']).dt.days.abs()
        df_merge.drop(columns=['hearing_date','ticket_issued_date'], inplace=True)
        
        if isTrain:
            # Include compliance in training data
            df = df.set_index('ticket_id')
            df_merge['compliance'] = df['compliance']
            # Remove nan values
            df_merge.dropna(subset=['compliance', 'lat', 'lon'], axis=0, inplace=True)
        
        return df_merge
    
    df_train = preprocess(df_train)
    X_test = preprocess(df_test, isTrain=False)
    
    X = df_train.drop(columns='compliance')
    y = df_train['compliance']
    
    # Split data for training and test validation
    X_train, X_test_validation, y_train, y_test_validation = train_test_split(X, y, random_state=0)
    
    # Fill nan values
    X_train.loc[:,'diff_date'].fillna(X_train.loc[:,'diff_date'].mean(), inplace=True)
    X_test_validation.loc[:, 'diff_date'].fillna(X_test_validation.loc[:,'diff_date'].mean(), inplace=True)
    X_test.loc[:, 'diff_date'].fillna(X_test.loc[:,'diff_date'].mean(), inplace=True)
    X_test.loc[:, 'lat'].fillna(X_test.loc[:, 'lat'].mean(), inplace=True)
    X_test.loc[:, 'lon'].fillna(X_test.loc[:, 'lon'].mean(), inplace=True)
    
    clf = RandomForestClassifier(n_estimators=50)
    param_grid = {
        'max_depth': [80, 90, 100],
        'max_features': [2, 3],
    }
    
    # Instantiate the grid search model
    grid_search = GridSearchCV(estimator=clf, param_grid=param_grid, cv=3,
                               n_jobs=-1, verbose=2, scoring='roc_auc')
    
    grid_search.fit(X_train, y_train)
    print('GridSearch best param: ', grid_search.best_params_)
    
    y_score_clf = grid_search.predict(X_test_validation)
    fpr_clf, tpr_clf, _ = roc_curve(y_test_validation, y_score_clf)
    roc_auc_clf = auc(fpr_clf, tpr_clf)
    print(roc_auc_clf)
    
    X_test['compliance'] = grid_search.predict_proba(X_test)[:,1]
    
    return X_test['compliance'].astype('float32')

blight_model()

Fitting 3 folds for each of 6 candidates, totalling 18 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 out of  18 | elapsed:  1.2min finished


GridSearch best param:  {'max_depth': 100, 'max_features': 3}
0.6389092056622485


ticket_id
284932    0.02
285362    0.10
285361    0.00
285338    0.26
285346    0.02
285345    0.14
285347    0.00
285342    0.30
285530    0.00
284989    0.00
285344    0.00
285343    0.00
285340    0.02
285341    0.04
289828    0.00
289830    0.04
289829    0.14
292133    0.02
292134    0.04
285349    0.14
285348    0.02
284991    0.00
285532    0.20
286073    0.38
285406    0.06
285001    0.00
285006    0.00
365862    0.94
285405    0.12
287857    0.00
          ... 
376276    0.00
376218    0.06
376368    0.12
376369    0.02
376225    0.00
376222    0.00
376362    0.00
376363    0.10
376228    0.40
376265    0.08
376286    0.10
376320    0.02
376314    0.04
376327    0.30
376435    0.54
376434    0.02
376459    0.00
376478    0.00
376473    0.00
376484    0.00
376482    0.06
376480    0.06
376479    0.06
376481    0.06
376483    0.02
376496    0.00
376497    0.00
376499    0.00
376500    0.00
369851    0.08
Name: compliance, Length: 61001, dtype: float32